## Attempt using a simple reranking pipeline

In [2]:
import os
import pyterrier as pt
from tira.third_party_integrations import ensure_pyterrier_is_loaded
from pathlib import Path
import ir_datasets
from tqdm import tqdm
from tirex_tracker import tracking, ExportFormat
from sentence_transformers import CrossEncoder

In [3]:
output = Path("./output")

def run_tira_experiment(pipeline, pipeline_name, dataset_id, topics, description):
    target_dir = output / "runs" / dataset_id / pipeline_name
    target_file = target_dir / "run.txt.gz"

    if target_file.exists():
        print(f"Run existiert bereits: {target_dir}")
        return

    target_dir.mkdir(parents=True, exist_ok=True)

    print(f"--- Starte: {pipeline_name} ---")

    with tracking(
        export_file_path=target_dir / "ir-metadata.yml", 
        export_format=ExportFormat.IR_METADATA, 
        system_description=description, 
        system_name=pipeline_name
    ):
        run = pipeline.transform(topics)

    pt.io.write_results(run, str(target_file))
    print(f"Gespeichert in: {target_file}")

Load index and corpus

In [4]:
ensure_pyterrier_is_loaded()
ds_id = "radboud-validation-20251114-training"
dataset = pt.datasets.get_dataset(f"irds:ir-lab-wise-2025/{ds_id}")
topics = dataset.get_topics("title")
qrels = dataset.get_qrels()

Java started and loaded: pyterrier.java.colab, pyterrier.java, pyterrier.java.24, pyterrier.terrier.java [version=5.11 (build: craig.macdonald 2025-01-13 21:29), helper_version=0.0.8]


In [5]:
index_path = "./output/indexes/radboud-validation"
abs_index_path = os.path.abspath(index_path)

if not os.path.exists(abs_index_path + "/data.properties"):
    print("Building index...")
    irds_ds = ir_datasets.load(f"ir-lab-wise-2025/{ds_id}")
    
    def doc_iter():
        for doc in tqdm(irds_ds.docs_iter(), desc="Indexing"):
            yield {'docno': doc.doc_id, 'text': doc.default_text()}
            
    indexer = pt.IterDictIndexer(abs_index_path, meta={'docno': 100})
    index_ref = indexer.index(doc_iter())
    index = pt.IndexFactory.of(index_ref)
else:
    print("Loading existing index...")
    index = pt.IndexFactory.of(abs_index_path)

print(f"Index loaded: {index}")

Loading existing index...
Index loaded: <org.terrier.structures.Index at 0x76c0a9979ad0 jclass=org/terrier/structures/Index jself=<LocalRef obj=0x5e9e6a618650 at 0x76c0a9a67f50>>


Load models and piplines

In [6]:
irds_ds = ir_datasets.load(f"ir-lab-wise-2025/{ds_id}")

doc_text_map = {doc.doc_id: doc.default_text() for doc in tqdm(irds_ds.docs_iter(), desc="Loading Docs")}

def _lookup_text(row):
    return doc_text_map.get(row['docno'], "")

text_getter = pt.apply.text(_lookup_text)

Loading Docs: 63621it [00:23, 2684.09it/s]


In [7]:
model = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-6-v2', device="cpu")
bm25 = pt.BatchRetrieve(index, wmodel="BM25")

def _cross_encoder_score(row):
    return float(model.predict([(row['query'], row['text'])])[0])

/tmp/ipykernel_587/2845150464.py:2: DeprecationWarning: Call to deprecated class BatchRetrieve. (use pt.terrier.Retriever() instead) -- Deprecated since version 0.11.0.
  bm25 = pt.BatchRetrieve(index, wmodel="BM25")


In [8]:
get_text = pt.apply.text(_lookup_text)
reranker = pt.apply.doc_score(_cross_encoder_score)

pipeline_simple_rerank = (
    bm25 % 50
    >> get_text
    >> reranker
)

Run experiments

In [9]:
print("Starte Simple Re-Ranking Experiment...")
pt.Experiment(
    [bm25, pipeline_simple_rerank],
    topics,
    qrels,
    eval_metrics=["map", "ndcg_cut_10", "P_10"],
    names=["BM25 Baseline", "BM25 + Cross-Encoder (Top 50)"],
    verbose=True
)

Starte Simple Re-Ranking Experiment...


pt.Experiment: 100%|██████████| 2/2 [02:15<00:00, 67.77s/system]


,name,map,P_10,ndcg_cut_10
0,BM25 Baseline,0.381395,0.432143,0.451635
1,BM25 + Cross-Encoder (Top 50),0.388355,0.485714,0.521048


In [11]:
run_tira_experiment(
    pipeline=pipeline_simple_rerank,
    pipeline_name="simple_rerank",
    dataset_id=ds_id,
    topics=topics,
    description="Standard BM25 retrieval with ms-marco reranker."
)

--- Starte: simple_rerank ---


Detected a hypervisor/virtualization technology. Some metrics might not be available due to configuration or availability of virtual hardware features.

=====  Processor information  =====
Linux arch_perfmon flag  : yes
Hybrid processor         : no
IBRS and IBPB supported  : yes
STIBP supported          : yes
Spec arch caps supported : yes
Max CPUID level          : 27
CPU model number         : 140
PCM Error: can't open MSR handle for core 0 (No such file or directory)
Try no-MSR mode by setting env variable PCM_NO_MSR=1
Can not access CPUs Model Specific Registers (MSRs).
execute 'modprobe msr' as root user, then execute pcm as root user.


Gespeichert in: output/runs/radboud-validation-20251114-training/simple_rerank/run.txt.gz
